In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split,GridSearchCV, cross_val_score 
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.linear_model import Ridge,Lasso
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score,roc_curve,roc_auc_score,accuracy_score,r2_score,mean_absolute_error,mean_squared_error
import warnings
warnings.filterwarnings('ignore')
#for display all rows-columns
#pd.set_option('display.max_rows',None)
pd.pandas.set_option('display.max_columns',None)

In [ ]:
df=pd.read_csv(r"D:\OneDrive\Documents\Downloads\Intern\Project-Housing_splitted\train.csv")
df

In [ ]:
pd.set_option('display.max_rows',None)

In [ ]:
df.head()

In [ ]:
df.dtypes

In [ ]:
df.info()

In [ ]:
df.isnull().sum()

In [ ]:
## here we find the percentage to nan value in each features
# step 1- the features which has missing value
# step 2- print the features name with the percentage
features_with_na=[features for features in df.columns if df[features].isnull().sum()>1]  #  1 is for -- if atleast 1 na define.
for feature in features_with_na:
    print(feature,np.round(df[feature].isnull().mean(),2),"Missing value") # 2 HERE FOR DECIMAL VALUE

# since it has missing values, we need to find relationship between  missing value and target (label).

## Let's plot the graph

In [ ]:
for feature in features_with_na:
#let's make the variable that indicates 1 if the observation is missing
    feature=np.where(df[feature].isnull(),1 , 0)
#let's calculate the mean value of SalesPrice where the info is missing
    df.groupby(feature)["SalePrice"].median().plot.bar()
    plt.title(feature)
    plt.show()

#### Here is clearly visible relationship of nan value with SalePrice(Depandent variable), so we need to replace these nan value with meaningfull meaning.

# Numerical variable

 Here we seperated Numerical Features from whole dataset.

In [ ]:
# list of numerical variable
Numerical_feature=[feature for feature in df.columns if df[feature].dtypes!='O']  # o means object
print('Numerical_features',len(Numerical_feature))
df[Numerical_feature].head()

'''After looking the dataset we found here some year data. we have exact info of data-time variable like no. of year, month of year like example when we sold the house and build the house.'''

In [ ]:
df[Numerical_feature].isnull().sum()

In [ ]:
year_feat=[feature for feature in Numerical_feature if 'Yr' in feature or 'Year' in feature]
print(year_feat)

In [ ]:
## Temoral variable(date time variables)
for feature in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    df[feature]=df['YrSold']-df[feature]

In [ ]:
df[['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']].head()

In [ ]:
for feature in year_feat:
    print(feature,df[feature].unique())

'''lets analyse the above year data'''

In [ ]:
df.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('yearsold')
plt.ylabel("median house price")
plt.title("house price vs Year sold price")

'''here as we see in above plot price is decresing way.'''

In [ ]:
df.groupby('YearBuilt')['SalePrice'].median().plot()
plt.xlabel('YearBuilt')
plt.ylabel("median house price")
plt.title("house price vs YearBuilt")

In [ ]:
# Here we will compare the difference between all years feature with SalePrice.
for feature in year_feat:
    if feature != 'YrSold':                                         # Repeat Again
        df[feature]=df["YrSold"]-df[feature]
        plt.scatter(df[feature],df['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel("SalePrice")
        plt.show()

In [ ]:
# Here we checked the Relationship Year feature 

In [ ]:
for feature in year_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

### Here we use box plot for checking the outliers in Year features

In [ ]:
### Numerical variable are usually of 2 type
# 1. Continous variable and Discreate variable
Discreate_feat=[feature for feature in Numerical_feature if len(df[feature].unique())<25 and feature not in year_feat+["Id"]]
Discreate_feat

In [ ]:
df[Discreate_feat]

In [ ]:
df[Discreate_feat].head()

In [ ]:
df[Discreate_feat].isnull().sum()

In [ ]:
for feature in Discreate_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
df[Discreate_feat].head()

In [ ]:
df[Discreate_feat].isnull().sum()

In [ ]:
for feature in Discreate_feat:
    df.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()
    

## we checked with relationship with bar plot among discreate feature with target label

# Countinous Variable

In [ ]:
conti_feat=[feature for feature in Numerical_feature if feature not in Discreate_feat+year_feat+["Id"]]
df[conti_feat]

In [ ]:
len(conti_feat)

In [ ]:
df[conti_feat].isnull().sum()

In [ ]:
for feature in conti_feat:
    df[feature].hist(bins=15)
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.title(feature)
    plt.show()

# Exploratory data analysis`

In [ ]:
# here we are using logarithmic transform

for feature in conti_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
#         df['SalePrice']=np.log(df['SalePrice'])            #check this line 
        plt.scatter(df[feature],df["SalePrice"])
        plt.xlabel(feature)
        plt.ylabel("SalePrice")
        plt.title(feature)
        plt.show()

## Same as above plot  correlation with scatter plot in continous data.

In [ ]:
df[conti_feat]

In [ ]:
for feature in conti_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

# Categorical Features

In [ ]:
cat_feat=[feature for feature in df.columns if df[feature].dtypes=="O"]
df[cat_feat]

In [ ]:
for feature in cat_feat:
    print('The feature is {} and no. of categories are {}'.format(feature,len(df[feature].unique())))


In [ ]:
for feature in cat_feat:
    print('The feature is {} and no. of categories are {}'.format(feature,df[feature].unique()))

In [ ]:
# for find out the relationship between categorical and target

for feature in cat_feat:
    df.groupby(feature)['SalePrice'].median().plot.bar()
    plt.xlabel(feature)
    plt.ylabel('SalePrice')
    plt.title(feature)
    plt.show()

In [ ]:
cat_with_na=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes=='O']
for feature in cat_with_na:
    print('{}:{}%missing values'.format(feature,np.round(df[feature].isnull().mean(),2)))

### as above we found missing value in categorical data in percentage and below we replaced nan value with "missing" value/.

In [ ]:
def replace_cat_nan(df,cat_with_na):
    df[cat_with_na]=df[cat_with_na].fillna("Missing")
    return df
df=replace_cat_nan(df,cat_with_na)
df[cat_with_na].isnull().sum() 

In [ ]:
df[cat_with_na]

In [ ]:
conti_with_nan=[features for features in df.columns if df[features].dtypes!="O"]  #  1 is for -- if atleast 1 na define.
for feature in conti_with_nan:
    print(feature,np.round(df[feature].isnull().mean(),2),"Missing value") # 2 HERE FOR DECIMAL VALUE

In [ ]:
df[conti_with_nan].isnull().sum()

In [ ]:
Numerical_feature=[feature for feature in df.columns if df[feature].dtypes!='O']

In [ ]:
df[Numerical_feature]

In [ ]:
df[Numerical_feature].isnull().sum()

In [ ]:
Numerical_feat=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes!='O']

In [ ]:
df[Numerical_feat]

In [ ]:
#Replacing the Numerical Missing values
for feature in Numerical_feat:
    median_value=df[feature].median()
    df[feature].fillna(median_value,inplace=True)
df[Numerical_feat].isnull().sum()

In [ ]:
# from scipy.stats import zscore
# Numerical_feature=[feature for feature in df.columns if df[feature].dtypes!='O']
# for feature in Numerical_feature:
#     z=np.abs(zscore(df[Numerical_feature]))  
#     threshold=3                                                                #testing for loop for outliers
#     np.where(z>3)
#     df[feature]=Numerical_feature[(z<3).all(axis=1)]
#     df[feature]

In [ ]:
df.isnull().sum()

In [ ]:
df.columns

In [ ]:
plt.figure(figsize=(20,15))
plotnumber=1
for i in df[Numerical_feature]:
    if plotnumber <=36:
        ax=plt.subplot(12,3,plotnumber)
        sns.histplot(x=i,data=df[Numerical_feature])
        plt.title(i)
        plt.xlabel(i)
    plotnumber+=1
plt.show()

In [ ]:
#from scipy.stats import zscore
# z_score=zscore(df[['LotArea', 'LotFrontage']])
# z_score=np.abs(z_score)                                            #It worked. for removing outliers
# df[['LotArea', 'LotFrontage']]=z_score[(z_score<3).all(axis=1)]

In [ ]:
# from scipy.stats import zscore
# lecode=LabelEncoder()
# #X=lecode.fit_transform(x)                  #testing for loop for outliers
# #X
# cate=cat.apply(LabelEncoder().fit_transform)
# cate

# For Removing skewness through log Transform 

In [ ]:
for feature in Numerical_feature:
    if 0 in df[feature].unique():
        pass
    else:
        df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
df.skew()

In [ ]:
cate_feature=[feature for feature in df.columns if df[feature].dtypes=='O']
df[cate_feature]

In [ ]:
for feature in cate_feature:
    temp=df.groupby(feature)['SalePrice'].count()/len(df)
    temp_df=temp[temp>0.01].index
    df[feature]=np.where(df[feature].isin(temp_df),df[feature],"Rare_var")
    df.reset_index()

In [ ]:
from sklearn.preprocessing import LabelEncoder
# cols=['MSZoning', 'Street', 'LotShape', 'LandContour', 'Utilities',
#        'LotConfig', 'LandSlope', 'Neighborhood', 'Condition1', 'Condition2',
#        'BldgType', 'HouseStyle', 'RoofStyle', 'RoofMatl', 'Exterior1st',
#        'Exterior2nd', 'MasVnrType', 'ExterQual', 'ExterCond', 'Foundation',
#        'BsmtQual', 'BsmtCond', 'BsmtExposure', 'BsmtFinType1', 'BsmtFinType2',
#        'Heating', 'HeatingQC', 'CentralAir', 'Electrical', 'KitchenQual',
#        'Functional', 'GarageType', 'GarageFinish', 'GarageQual', 'GarageCond',
#        'PavedDrive', 'SaleType', 'SaleCondition']

df[cate_feature]=df[cate_feature].apply(LabelEncoder().fit_transform)

In [ ]:
df[cate_feature]

In [ ]:
df.corr()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(df[Numerical_feature].corr(),annot=True,fmt='.2f')

In [ ]:
# Vif=pd.DataFrame()
# Vif['Vif']=[variance_inflation_factor(df,i) for i in range(df.shape[1])]
# Vif['features']=df.columns
# Vif

In [ ]:
Vif=pd.DataFrame()
Vif['Vif']=[variance_inflation_factor(df[Numerical_feature],i) for i in range(df[Numerical_feature].shape[1])]
Vif['features']=df[Numerical_feature].columns
Vif

In [ ]:
X=df.iloc[:,1:-1]
Y=df.iloc[:,-1]


In [ ]:
Y=df.iloc[:,-1]


In [ ]:
scaler=MinMaxScaler()
x_scaled = scaler.fit_transform(X)
x_scaled

In [ ]:
x_train,x_test,y_train,y_test=train_test_split(x_scaled,Y,test_size=.25,random_state=15)

In [ ]:
print(x_train.shape)
print(y_train.shape)

In [ ]:
lr = LinearRegression()
lr.fit(x_scaled,Y)

In [ ]:
pred=lr.predict(x_test)
pred

In [ ]:
lr.score(x_train,y_train)

In [ ]:
lr.score(x_test, y_test)

In [ ]:
print("MsE",mean_squared_error(y_test,pred))

In [ ]:
print("MAE",mean_absolute_error(y_test,pred))

In [ ]:
 # Regularization with grid search CV

In [ ]:
#create and fit a ridge regression model testing each alpha
parameters={'alpha':[0.1,0.01,0.001,0.0001,1,0],'random_state':list(range(0,10))}
model=Ridge()
grid=GridSearchCV(model,parameters)
grid.fit(x_scaled,Y)
print(grid.best_params_)

In [ ]:
model=Ridge(alpha= 1, random_state=0)
model.fit(x_scaled,Y)

In [ ]:
prediction_Rid=model.predict(x_test)
prediction_Rid

In [ ]:
score_rid=model.score(x_scaled,Y)
score_rid

In [ ]:
mod=Lasso()
parameter={'alpha':[0.1,0.01,0.001,0.0001,1,0],'random_state':list(range(0,10))}
grid=GridSearchCV(mod,parameter)
grid.fit(x_scaled,Y)
print(grid.best_params_)

In [ ]:
mod=Lasso(alpha= 0.001, random_state=0)
mod.fit(x_scaled,Y)

In [ ]:
prediction_lasso=mod.predict(x_test)
prediction_lasso

In [ ]:
score_lasso=model.score(x_scaled,Y)
score_lasso


In [ ]:
print(grid.best_params_)

In [ ]:
cross_val= cross_val_score(lr,x_scaled,Y,cv=5)
print(cross_val)
print(cross_val.mean())
print(cross_val.std())

In [ ]:
#prediction through graph
plt.scatter(y_test,pred)
plt.xlabel('actual')
plt.ylabel('predicted')
plt.title('actual vs predicted')
plt.show()

In [ ]:
#for saving the file
import joblib
joblib.dump(lr,"House_prediction_price_Train.csv.obj")

In [ ]:
lr_joblib=joblib.load("House_prediction_price_Train.csv.obj")
lr_joblib

In [ ]:
df=pd.read_csv(r"D:\OneDrive\Documents\Downloads\Intern\Project-Housing_splitted\train.csv")
df

In [ ]:
df.head()


In [ ]:
df.dtypes


In [ ]:
df.info()


In [ ]:
df.isnull().sum()

In [ ]:
year_feat=[feature for feature in Numerical_feature if 'Yr' in feature or 'Year' in feature]
print(year_feat)

In [ ]:
features_with_na=[features for features in df.columns if df[features].isnull().sum()>1]  #  1 is for -- if atleast 1 na define.
for feature in features_with_na:
    print(feature,np.round(df[feature].isnull().mean(),2),"Missing value")

In [ ]:
for feature in features_with_na:
#let's make the variable that indicates 1 if the observation is missing
    feature=np.where(df[feature].isnull(),1 , 0)
#let's calculate the mean value of SalesPrice where the info is missing
    df.groupby(feature)["SalePrice"].median().plot.bar()
    plt.title(feature)
    plt.show()

In [ ]:
# list of numerical variable
Numerical_feature=[feature for feature in df.columns if df[feature].dtypes!='O']  # o means object
print('Numerical_features',len(Numerical_feature))
df[Numerical_feature].head()

In [ ]:
df[Numerical_feature].isnull().sum()
year_feat=[feature for feature in Numerical_feature if 'Yr' in feature or 'Year' in feature]
print(year_feat)


In [ ]:
## Temoral variable(date time variables)
for feature in ['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']:
    df[feature]=df['YrSold']-df[feature]


In [ ]:
df[['YearBuilt', 'YearRemodAdd', 'GarageYrBlt']].head()
for feature in year_feat:
    print(feature,df[feature].unique())


In [ ]:
df.groupby('YrSold')['SalePrice'].median().plot()
plt.xlabel('yearsold')
plt.ylabel("median house price")
plt.title("house price vs Year sold price")

In [ ]:
df.groupby('YearBuilt')['SalePrice'].median().plot()
plt.xlabel('YearBuilt')
plt.ylabel("median house price")
plt.title("house price vs YearBuilt")

In [ ]:
# Here we will compare the difference between all years feature with SalePrice.
for feature in year_feat:
    if feature != 'YrSold':                                         # Repeat Again
        df[feature]=df["YrSold"]-df[feature]
        plt.scatter(df[feature],df['SalePrice'])
        plt.xlabel(feature)
        plt.ylabel("SalePrice")
        plt.show()

In [ ]:
for feature in year_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
### Numerical variable are usually of 2 type
# 1. Continous variable and Discreate variable
Discreate_feat=[feature for feature in Numerical_feature if len(df[feature].unique())<25 and feature not in year_feat+["Id"]]
Discreate_feat

In [ ]:
df[Discreate_feat].isnull().sum()

In [ ]:
for feature in Discreate_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
conti_feat=[feature for feature in Numerical_feature if feature not in Discreate_feat+year_feat+["Id"]]
df[conti_feat]

In [ ]:
df[conti_feat].isnull().sum()

In [ ]:
for feature in conti_feat:
    df[feature].hist(bins=15)
    plt.xlabel(feature)
    plt.ylabel('count')
    plt.title(feature)
    plt.show()

In [ ]:
# here we are using logarithmic transform

for feature in conti_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
#         df['SalePrice']=np.log(df['SalePrice'])            #check this line 
        plt.scatter(df[feature],df["SalePrice"])
        plt.xlabel(feature)
        plt.ylabel("SalePrice")
        plt.title(feature)
        plt.show()

In [ ]:
for feature in conti_feat:
    if 0 in df[feature].unique():
        pass
    else:
#         df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
for feature in cat_feat:
    print('The feature is {} and no. of categories are {}'.format(feature,len(df[feature].unique())))


In [ ]:
cat_feat=[feature for feature in df.columns if df[feature].dtypes=="O"]
df[cat_feat]

In [ ]:
cat_with_na=[feature for feature in df.columns if df[feature].isnull().sum()>1 and df[feature].dtypes=='O']
for feature in cat_with_na:
    print('{}:{}%missing values'.format(feature,np.round(df[feature].isnull().mean(),2)))

In [ ]:
def replace_cat_nan(df,cat_with_na):
    df[cat_with_na]=df[cat_with_na].fillna("Missing")
    return df
df=replace_cat_nan(df,cat_with_na)
df[cat_with_na].isnull().sum() 

In [ ]:
conti_with_nan=[features for features in df.columns if df[features].dtypes!="O"]  #  1 is for -- if atleast 1 na define.
for feature in conti_with_nan:
    print(feature,np.round(df[feature].isnull().mean(),2),"Missing value") # 2 HERE FOR DECIMAL VALUE

In [ ]:
Numerical_feature=[feature for feature in df.columns if df[feature].dtypes!='O']

In [ ]:
#Replacing the Numerical Missing values
for feature in Numerical_feat:
    median_value=df[feature].median()
    df[feature].fillna(median_value,inplace=True)
df[Numerical_feat].isnull().sum()

In [ ]:
for feature in Numerical_feature:
    if 0 in df[feature].unique():
        pass
    else:
        df[feature]=np.log(df[feature])
        df.boxplot(column=feature)
        plt.ylabel(feature)
        plt.title(feature)
        plt.show()

In [ ]:
df.skew()

In [ ]:
from sklearn.preprocessing import LabelEncoder
df[cate_feature]=df[cate_feature].apply(LabelEncoder().fit_transform)

In [ ]:
Vif=pd.DataFrame()
Vif['Vif']=[variance_inflation_factor(df[Numerical_feature],i) for i in range(df[Numerical_feature].shape[1])]
Vif['features']=df[Numerical_feature].columns
Vif

In [ ]:
X=df.iloc[:,1:-1]

In [ ]:
scaler=MinMaxScaler()
x_scaled = scaler.fit_transform(X)
x_scaled

In [ ]:
#for saving the file
import joblib
joblib.dump(lr,"House_prediction_price_test.csv.obj")

In [ ]:
x_test = pd.DataFrame(x_scaled)
x_test

In [ ]:
Test=lr_joblib.predict(x_test)

In [ ]:
Test

In [ ]:
df=pd.DataFrame(Test)
df

In [ ]:
#for saving the file
import joblib
joblib.dump(lg,"Test.csv.obj")